# Model calibration
This notebook contains a calibration of the model for the current number of agents. Usually ABMs are not linearly scalable and therefore this procedure should be followed for model calibration, to avoid unnecessary long start-up time of the model and thus a wastage of computing power. Because the RIVM stakeholders mentioned during one of the final meetings that they were especially interested in the process of model-building, this notebook is an annex to assist for when the model is built. Note that in case this model will be extended, this notebook will require to be extended, since it only includes current variables. However, most certainly the process stays the same.

Calibration outcomes of the model need to be implemented directly in the agents manually. Due to ABMs being stochastic, initial values will not be perfect but provide a good starting point. First we will be importing the model in the notebook and set it up in such a way that all policies and scenarios are turned off to see whether we can get the model in some sort of equilibrium.

In [1]:
from model.cepai_model import *

levers = {
    "L1": 0.0,  # Minimal requirement for reused parts
    "L2": 0.0,  # Minimal requirement for high-quality plastic
    "L3": 0.0,  # Use better solvable cohesives
    "L4": 1.0,  # Include externality for virgin plastic
    "L5": 0.05   # Minimal requirement for recyclate, calibrated for 5% according to Volvo.
        }

uncertainties = {
    "X1": 0.0,  # Annual increase factor of oil price
    "X2": 0.0,  # Annual probability for global oil shock
    "X3": 0.0   # Annual increase of recycling efficiency
        }

In [2]:
model = CEPAIModel(levers=levers, uncertainties=uncertainties)

Simulation starting...


In [3]:
model_run = model.run(steps=50, time_tracking=True)


KeyboardInterrupt



In [ ]:
model_run

In [5]:
for _ in range(50):
    model.step()

In [12]:
for PM in model.all_agents[PartsManufacturer]:
    print(PM.demand)

{<Component.VIRGIN: 1>: 53.08963373130812, <Component.RECYCLATE_LOW: 2>: 2.910366268691884, <Component.RECYCLATE_HIGH: 3>: 0.0, <Component.PARTS: 4>: 56}
{<Component.VIRGIN: 1>: 4888.775639518134, <Component.RECYCLATE_LOW: 2>: 270.22436048186586, <Component.RECYCLATE_HIGH: 3>: 0.0, <Component.PARTS: 4>: 5159}
{<Component.VIRGIN: 1>: 51.934324748281, <Component.RECYCLATE_LOW: 2>: 3.0656752517189965, <Component.RECYCLATE_HIGH: 3>: 0.0, <Component.PARTS: 4>: 55}


In [9]:
for CM in model.all_agents[CarManufacturer]:
    # print(CM.stock)
    print(CM.demand)
    print(CM.sold_volume)

{<Component.VIRGIN: 1>: 0.0, <Component.RECYCLATE_LOW: 2>: 0.0, <Component.RECYCLATE_HIGH: 3>: 0.0, <Component.PARTS: 4>: 32, <Component.CARS: 6>: 9}
{'last': 9, 'second_last': 9}
{<Component.VIRGIN: 1>: 0.0, <Component.RECYCLATE_LOW: 2>: 0.0, <Component.RECYCLATE_HIGH: 3>: 0.0, <Component.PARTS: 4>: 48, <Component.CARS: 6>: 14}
{'last': 14, 'second_last': 14}
{<Component.VIRGIN: 1>: 0.0, <Component.RECYCLATE_LOW: 2>: 0.0, <Component.RECYCLATE_HIGH: 3>: 0.0, <Component.PARTS: 4>: 28, <Component.CARS: 6>: 9}
{'last': 9, 'second_last': 9}
{<Component.VIRGIN: 1>: 0.0, <Component.RECYCLATE_LOW: 2>: 0.0, <Component.RECYCLATE_HIGH: 3>: 0.0, <Component.PARTS: 4>: 28, <Component.CARS: 6>: 6}
{'last': 6, 'second_last': 6}


In [8]:
model.step()

In [31]:
counter = 0
for agent in model.all_agents[User]:
    if agent.stock[Component.CARS]:
        counter += 1
        car = agent.stock[Component.CARS][0]
        print(car.state)
    else:
        print(agent.demand[Component.CARS])

CarState.FUNCTIONING
CarState.FUNCTIONING
CarState.BROKEN
CarState.BROKEN
1
CarState.FUNCTIONING
CarState.FUNCTIONING
CarState.FUNCTIONING
CarState.BROKEN
CarState.FUNCTIONING
CarState.FUNCTIONING
CarState.FUNCTIONING
CarState.BROKEN
CarState.FUNCTIONING
CarState.FUNCTIONING
CarState.FUNCTIONING
1
CarState.BROKEN
CarState.FUNCTIONING
1
CarState.FUNCTIONING
CarState.END_OF_LIFE
CarState.FUNCTIONING
CarState.FUNCTIONING
CarState.FUNCTIONING
CarState.FUNCTIONING
CarState.FUNCTIONING
CarState.FUNCTIONING
CarState.FUNCTIONING
CarState.FUNCTIONING
CarState.FUNCTIONING
CarState.FUNCTIONING
0
1
CarState.FUNCTIONING
CarState.FUNCTIONING
CarState.FUNCTIONING
CarState.FUNCTIONING
CarState.FUNCTIONING
CarState.FUNCTIONING
CarState.FUNCTIONING
CarState.FUNCTIONING
CarState.FUNCTIONING
CarState.FUNCTIONING
CarState.FUNCTIONING
CarState.BROKEN
CarState.FUNCTIONING
CarState.BROKEN
CarState.FUNCTIONING
CarState.BROKEN
CarState.FUNCTIONING
CarState.FUNCTIONING
CarState.BROKEN
CarState.FUNCTIONING
CarSta

In [7]:
print(counter)

939


In [33]:
len(model.all_agents[User])

1000

In [9]:
cars_tb_repaired = 0
for garage in model.all_agents[Garage]:
    cars_tb_repaired += len(garage.stock[Component.CARS])
    print("Cars tb repaired")
    print(garage.stock[Component.CARS])
    print("Cars tb recycled")
    print(garage.stock[Component.CARS])
    print("Cars tb dismantled")
    print(garage.stock[Component.CARS])

Cars tb repaired
[<model.bigger_components.Car object at 0x0000025D78E7DA50>, <model.bigger_components.Car object at 0x0000025D78E7DC30>, <model.bigger_components.Car object at 0x0000025D78E7E3B0>, <model.bigger_components.Car object at 0x0000025D78E7F8B0>, <model.bigger_components.Car object at 0x0000025D78EAD750>, <model.bigger_components.Car object at 0x0000025D78EAF970>, <model.bigger_components.Car object at 0x0000025D78EE19F0>, <model.bigger_components.Car object at 0x0000025D78EE39D0>, <model.bigger_components.Car object at 0x0000025D78FF53F0>, <model.bigger_components.Car object at 0x0000025D78F12560>, <model.bigger_components.Car object at 0x0000025D78FF5420>, <model.bigger_components.Car object at 0x0000025D78B87310>, <model.bigger_components.Car object at 0x0000025D788BE3B0>, <model.bigger_components.Car object at 0x0000025D78FB3A90>, <model.bigger_components.Car object at 0x0000025D78FB3B80>, <model.bigger_components.Car object at 0x0000025D78FB3790>, <model.bigger_componen

In [10]:
print(cars_tb_repaired)

52
